In [1]:
# Install Java, Spark, and Findspark
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/spark-2.4.6/spark-2.4.6-bin-hadoop2.7.tgz
!tar xf spark-2.4.6-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.6-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2020-08-04 04:06:18--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  1.05MB/s    in 0.8s    

2020-08-04 04:06:20 (1.05 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [4]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Apparel_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
apparel_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Apparel_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)

# Show DataFrame
apparel_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   32158956|R1KKOXHNI8MSXU|B01KL6O72Y|      24485154|Easy Tool Stainle...|         Apparel|          4|            0|          0|   N|                Y|★ THESE REALLY DO...|These Really Do W...|2013-01-14 00:00:00|
|         US|    2714559|R26SP2OPDK4HT7|B01ID3ZS5W|     363128556|V28 Women Cowl Ne...| 

In [5]:
# number of records in the dataset
apparel_df.count()

5906333

In [6]:
apparel_df.describe().show()

+-------+-----------+--------------------+--------------+-------------------+--------------------+--------------------+----------------+------------------+------------------+------------------+-------+-----------------+--------------------+--------------------+
|summary|marketplace|         customer_id|     review_id|         product_id|      product_parent|       product_title|product_category|       star_rating|     helpful_votes|       total_votes|   vine|verified_purchase|     review_headline|         review_body|
+-------+-----------+--------------------+--------------+-------------------+--------------------+--------------------+----------------+------------------+------------------+------------------+-------+-----------------+--------------------+--------------------+
|  count|    5906333|             5906333|       5906333|            5906333|             5906333|             5906333|         5906322|           5906322|           5906322|           5906322|5906322|          590

In [8]:
review_id_df = apparel_df.select(['review_id','customer_id','product_id','product_parent', 'review_date'])
review_id_df.show()

+--------------+-----------+----------+--------------+-------------------+
|     review_id|customer_id|product_id|product_parent|        review_date|
+--------------+-----------+----------+--------------+-------------------+
|R1KKOXHNI8MSXU|   32158956|B01KL6O72Y|      24485154|2013-01-14 00:00:00|
|R26SP2OPDK4HT7|    2714559|B01ID3ZS5W|     363128556|2014-03-04 00:00:00|
| RWQEDYAX373I1|   12608825|B01I497BGY|     811958549|2015-07-12 00:00:00|
|R231YI7R4GPF6J|   25482800|B01HDXFZK6|     692205728|2015-06-03 00:00:00|
|R3KO3W45DD0L1K|    9310286|B01G6MBEBY|     431150422|2015-06-12 00:00:00|
|R1C4QH63NFL5NJ|   26631939|B01FWRXN0Y|     366144407|2014-04-22 00:00:00|
|R2GP65O1U9N7BP|   48785098|B01EXNH1HE|     786052021|2015-07-28 00:00:00|
|R3O29CT5MQQ3XQ|   39548589|B01E7OL09O|     108920964|2015-07-10 00:00:00|
|R1ZECD2AA8QFF6|   29355866|B01DXHX81O|     317132458|2015-08-09 00:00:00|
|R2S79GCF6J89OA|   27477484|B01DDULIJK|     110598191|2014-05-24 00:00:00|
|R1CBCUGNP37MPK|   176858

In [9]:
products_df = apparel_df.select(['product_id','product_title']).distinct()
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|9822497415|Tommy Bahama Bird...|
|B00006AW85|John Deere Sweats...|
|B00006LFGJ|Women's Black Fle...|
|B00006THQ5|Roberto Coin Star...|
|B00006WMIV|Microfiber Contou...|
|B00006WRFU|Kenneth Cole Reac...|
|B00006X1WG|Undergear Cotton ...|
|B00006XJ1C|Silk Noile Trouse...|
|B00007EUVW|LEVIS SALT STAIN ...|
|B00007GBIJ|        Custom Rolex|
|B00007ID05|Deluxe Arctic Sur...|
|B0000863G5|108 Fgh40 Ecru Le...|
|B00008IOAK|Braza Reveal Clot...|
|B00008NRDU|           SWEET TEE|
|B00008NSOG|Reaction Kenneth ...|
|B00008ZGCO| UTILITY CARGO CAPRI|
|B00009KEIJ|Women's Mossimo E...|
|B00009YRTH|Thermal Crewneck ...|
|B0000ASFXX|Carhartt Men's Qu...|
|B0000CC55O|Youth M A1 Jacket...|
+----------+--------------------+
only showing top 20 rows



In [10]:
group_df = apparel_df.groupBy("customer_id").count()
# withColumnRenamed does not modify current dataFrame, instead it returns the modified version
customers_df = group_df.withColumnRenamed('count','customer_count')
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   13780696|             1|
|   48489393|             8|
|   43380682|             1|
|   17133428|             1|
|    9731896|             2|
|    1797768|             1|
|   27446622|             1|
|     120706|             3|
|   15915947|             2|
|   37520920|             1|
|   43920023|             9|
|   14309190|             8|
|   22316505|             1|
|   43949054|             2|
|   21102747|             1|
|   19021463|             1|
|   16075459|             8|
|   51475782|             2|
|   11555870|             5|
|   35471383|             3|
+-----------+--------------+
only showing top 20 rows



In [12]:
vine_df = apparel_df.select(['review_id','star_rating','helpful_votes','total_votes','vine'])
vine_df.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R1KKOXHNI8MSXU|          4|            0|          0|   N|
|R26SP2OPDK4HT7|          5|            1|          2|   N|
| RWQEDYAX373I1|          5|            0|          0|   N|
|R231YI7R4GPF6J|          5|            0|          0|   N|
|R3KO3W45DD0L1K|          5|            0|          0|   N|
|R1C4QH63NFL5NJ|          5|            0|          0|   N|
|R2GP65O1U9N7BP|          5|            0|          0|   N|
|R3O29CT5MQQ3XQ|          4|            0|          0|   N|
|R1ZECD2AA8QFF6|          5|            0|          0|   N|
|R2S79GCF6J89OA|          3|            0|          0|   N|
|R1CBCUGNP37MPK|          5|            0|          0|   N|
|R3NU4WUR5ZNV1V|          5|            0|          0|   N|
|R32EPCJ3XF8LGE|          5|            0|          0|   N|
|R1XIBC6WQBW31M|          5|            

In [18]:
# Configure settings for RDS data-bootcamp.crrqs5jt3ck3.us-east-1.rds.amazonaws.com
mode = "append"
jdbc_url="jdbc:postgresql://data-bootcamp.crrqs5jt3ck3.us-east-1.rds.amazonaws.com:5432/bigdatahomework"
config = {"user":"xxxxxxxx", 
          "password": "xxxxxxxx", 
          "driver":"org.postgresql.Driver"}


# write dataframe to tabels

In [19]:
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [20]:
products_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [21]:
customers_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [22]:
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)